## Preambule

In [150]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import scipy
from scipy.signal import savgol_filter

## Run classes

In [2]:
import class_datareading
reload(class_datareading)
from class_datareading import datareading

datareader = datareading()
datareader.read_general()
datareader.read_ssps()
datareader.read_undata()
datareader.read_historicalemis()
datareader.read_ar6()
# datareader.determine_global_budgets()
# datareader.determine_global_trajectories()
# datareader.read_baseline()
# datareader.read_ndc()
# datareader.merge_xr()
# datareader.add_country_groups()
# datareader.save()

# ==================================== #
# Initializing datareading class     #
# ==================================== #
- Reading general data
- Reading GDP and population data from SSPs
- Reading UN population data (for past population)
- Reading historical emissions (primap)
- Read AR6 data


In [3]:
self = datareader

In [ ]:
# CO2 budgets
df_budgets = pd.read_csv("X:/user/dekkerm/Data/Budgets_Forster2023/ClimateIndicator-data-ed37002/data/carbon_budget/updateMAGICCscen_temp2013_2022-budget.csv")
df_budgets = df_budgets[["dT_targets", "0.1", "0.17", "0.33", "0.5", "0.66", "0.83", '0.9']]
dummy = df_budgets.melt(id_vars=["dT_targets"], var_name="Probability", value_name="Budget")
ar = np.array(dummy['Probability'])
ar = ar.astype(float).round(2)
ar[ar == 0.66] = 0.67
dummy['Probability'] = ar
dummy['dT_targets'] = dummy['dT_targets'].astype(float).round(1)
dummy = dummy.set_index(["dT_targets", "Probability"])
dummy['Budget'] = dummy['Budget'] + float(self.xr_primap.sel(Region='EARTH', Time=2021).CO2_hist)/1e3 + 40.9 # from Forster 2023 for the year 2022 -> 1 Jan 2021 as starting year! So not + float(self.xr_primap.sel(Region='EARTH', Time=2020).CO2_hist)/1e3 
xr_bud_co2 = xr.Dataset.from_dataframe(dummy)
xr_bud_co2 = xr_bud_co2.rename({'dT_targets': "Temperature"}).sel(Temperature = [1.5, 1.7, 2.0])

# Interpolate
Blist = np.zeros(shape=(len(self.Tlist), len(self.Plist)))+np.nan
for p_i, p in enumerate(self.Plist):
    a, b = np.polyfit(xr_bud_co2.Temperature, xr_bud_co2.sel(Probability = np.round(p, 2)).Budget, 1)
    for t_i, t in enumerate(self.Tlist): # Interpolate only on the level of temperature
        Blist[t_i, p_i] = a*t+b
data2 = xr.DataArray(Blist,
                    coords={'Temperature': self.Tlist,
                            'Risk': (1-self.Plist).astype(float).round(2)},
                    dims=['Temperature', 'Risk'])
self.xr_co2_budgets = xr.Dataset({'Budget': data2})

In [243]:
# Non-CO2 trajectories
ch4_hist_2020 = self.xr_primap.sel(Region='EARTH').sel(Time=2020).CH4_hist
n2o_hist_2020 = self.xr_primap.sel(Region='EARTH').sel(Time=2020).N2O_hist*1e3

# Rescale CH4 and N2O trajectories
compensation_form = np.array(list(np.linspace(0, 1, len(np.arange(self.settings['params']['start_year_analysis'], 2050))))+[1]*len(np.arange(2050, 2101)))
xr_comp =  xr.DataArray(1-compensation_form, dims=['Time'], coords={'Time': np.arange(self.settings['params']['start_year_analysis'], 2101)})

xr_ch4_raw = self.xr_ar6.sel(Variable='Emissions|CH4', Time=np.arange(2021, 2101))
offset = xr_ch4_raw.sel(Time = 2021) - float(self.xr_primap.sel(Time=2021, Region='EARTH').CH4_hist)
xr_ch4 = (-xr_comp*offset+xr_ch4_raw)#*self.settings['params']['gwp_ch4']/1e3

xr_n2o_raw = self.xr_ar6.sel(Variable='Emissions|N2O', Time=np.arange(2021, 2101))
offset = xr_n2o_raw.sel(Time = 2021) - float(self.xr_primap.sel(Time=2021, Region='EARTH').N2O_hist)*1e3
xr_n2o = (-xr_comp*offset+xr_n2o_raw)#*self.settings['params']['gwp_n2o']/1e3

# Obtain TCRE-dependent reductions of non-CO2
df = pd.read_excel("X:/user/dekkerm/Data/Budgets_Forster2023/NonCO2.xlsx", sheet_name='CH4')
dummy = df.melt(id_vars=["Temperature"], var_name="Probability", value_name="Value")
dummy = dummy.set_index(["Temperature", "Probability"])
xr_ch4_red = xr.Dataset.from_dataframe(dummy)/1e2
xr_ch4_red = xr_ch4_red.reindex({'Probability': [0.1, 0.17, 0.33, 0.5, 0.67, 0.75, 0.83, 0.90]})
xr_ch4_red = xr_ch4_red.interpolate_na(dim='Probability')
xr_ch4_red = xr_ch4_red.reindex({"Probability": [0.17, 0.33, 0.5, 0.67, 0.83]})

df = pd.read_excel("X:/user/dekkerm/Data/Budgets_Forster2023/NonCO2.xlsx", sheet_name='N2O')
dummy = df.melt(id_vars=["Temperature"], var_name="Probability", value_name="Value")
dummy = dummy.set_index(["Temperature", "Probability"])
xr_n2o_red = xr.Dataset.from_dataframe(dummy)/1e2
xr_n2o_red = xr_n2o_red.reindex({'Probability': [0.1, 0.17, 0.33, 0.5, 0.67, 0.75, 0.83, 0.90]})
xr_n2o_red = xr_n2o_red.interpolate_na(dim='Probability')
xr_n2o_red = xr_n2o_red.reindex({"Probability": [0.17, 0.33, 0.5, 0.67, 0.83]})

# Construct non-CO2 pathways
temps = []
probs = []
ch4s = []
times = []
n2os = []
for p_i, p in enumerate(np.array(self.xr_co2_budgets.Risk)):
    for t_i, t in enumerate(np.array(self.xr_co2_budgets.Temperature)):
        ch4_red = xr_ch4_red.sel(Temperature=t, Probability=np.round(1-p,2)).Value#[0.48, 0.47, 0.35][t_i]
        n2o_red = xr_n2o_red.sel(Temperature=t, Probability=np.round(1-p,2)).Value
        ch4_hist_2050 = ch4_hist_2020*(1+ch4_red)
        n2o_hist_2050 = n2o_hist_2020*(1+n2o_red)
        traj_ch4 = 1e-3*self.settings['params']['gwp_ch4']*(xr_ch4.sel(Time=np.arange(2021, 2101), ModelScenario = xr_ch4.ModelScenario[np.where(np.abs(xr_ch4.sel(Time=2050).Value - ch4_hist_2050) < ch4_hist_2050*0.1)[0]]).mean(dim='ModelScenario').Value)
        traj_n2o = 1e-6*self.settings['params']['gwp_n2o']*(xr_n2o.sel(Time=np.arange(2021, 2101), ModelScenario = xr_n2o.ModelScenario[np.where(np.abs(xr_n2o.sel(Time=2050).Value - n2o_hist_2050) < n2o_hist_2050*0.1)[0]]).mean(dim='ModelScenario').Value)
        for time_i, time in enumerate(np.arange(2021, 2101)):
            probs.append(p)
            temps.append(t)
            ch4s.append(float(traj_ch4[time_i]))
            n2os.append(float(traj_n2o[time_i]))
            times.append(time)

dict_ch4 = {}
dict_ch4['Temperature'] = temps
dict_ch4['Risk'] = 1-np.array(probs)
dict_ch4['Time'] = times
dict_ch4['Value'] = ch4s
df_ch4 = pd.DataFrame(dict_ch4)
dummy = df_ch4.set_index(["Temperature", "Risk", "Time"])
xr_traj_ch4 = xr.Dataset.from_dataframe(dummy)

dict_n2o = {}
dict_n2o['Temperature'] = temps
dict_n2o['Risk'] = 1-np.array(probs)
dict_n2o['Time'] = times
dict_n2o['Value'] = n2os
df_n2o = pd.DataFrame(dict_n2o)
dummy = df_n2o.set_index(["Temperature", "Risk", "Time"])
xr_traj_n2o = xr.Dataset.from_dataframe(dummy)

self.xr_traj_nonco2 = xr_traj_ch4+xr_traj_n2o

In [265]:
# CO2 trajectories

# Determine the trajectory
startpoint = self.xr_primap.sel(Time=self.settings['params']['start_year_analysis'], Region="EARTH").CO2_hist
compensation_form = np.array(list(np.linspace(0, 1, len(np.arange(self.settings['params']['start_year_analysis'], 2050))))+[1]*len(np.arange(2050, 2101)))
xr_comp =  xr.DataArray(compensation_form, dims=['Time'], coords={'Time': np.arange(self.settings['params']['start_year_analysis'], 2101)})

# Create an empty xarray.Dataset
xr_traj_co2 = xr.Dataset(
    coords={
        'NegEmis': self.Neglist,
        'TrajUnc': ['Earliest', 'Early', 'Medium', 'Late', 'Latest'],
        'Temperature': self.Tlist,
        'Risk': self.Plist,
        'Time': np.arange(self.settings['params']['start_year_analysis'], 2101),
    }
)

# Initialize data arrays for each variable
pathways_data = {
    'CO2_globe': xr.DataArray(
        data=np.nan,
        coords=xr_traj.coords,
        dims=('NegEmis', 'TrajUnc', 'Temperature', 'Risk', 'Time'),
        attrs={'description': 'Pathway data'}
    )
}

xr_scen2_use = self.xr_ar6.sel(Variable='Emissions|CO2')
xr_scen2_use = xr_scen2_use.reindex(Time = np.arange(2000, 2101, 10))
xr_scen2_use = xr_scen2_use.reindex(Time = np.arange(2000, 2101))
xr_scen2_use = xr_scen2_use.interpolate_na(dim="Time", method="linear")
xr_scen2_use = xr_scen2_use.reindex(Time = np.arange(self.settings['params']['start_year_analysis'], 2101))
totalnet = xr_scen2_use.sel(Time=np.arange(self.settings['params']['start_year_analysis'], 2101)).sum(dim='Time')/1e3
co2_start = xr_scen2_use.sel(Time=self.settings['params']['start_year_analysis'])/1e3
offsets = (startpoint/1e3-co2_start)
emis_all = xr_scen2_use.sel(Time=np.arange(self.settings['params']['start_year_analysis'], 2101))/1e3 + offsets
totalnets_corr = emis_all.sel(Time=np.arange(self.settings['params']['start_year_analysis'], 2101)).sum(dim='Time')

xr_scen2_neg = self.xr_ar6.sel(Variable=['Carbon Sequestration|CCS',
                                        'Carbon Sequestration|Land Use']).sum(dim='Variable')
xr_scen2_neg = xr_scen2_neg.reindex(Time = np.arange(2000, 2101, 10))
xr_scen2_neg = xr_scen2_neg.reindex(Time = np.arange(2000, 2101))
xr_scen2_neg = xr_scen2_neg.interpolate_na(dim="Time", method="linear")
xr_scen2_neg = xr_scen2_neg.reindex(Time = np.arange(self.settings['params']['start_year_analysis'], 2101))
totalneg = xr_scen2_neg.sel(Time=np.arange(2080, 2101)).mean(dim='Time')

for temp_i, temp in enumerate(self.Tlist):
    for risk_i, risk in enumerate(self.Plist):
        budget = self.xr_co2_budgets.Budget.sel(Temperature=temp, Risk=risk)
        ms1 = self.xr_ar6.ModelScenario[np.where(np.abs(totalnets_corr.Value - budget) < budget*0.2)[0]]
    
        # The 90-percentile
        totalneg_i = totalneg.sel(ModelScenario=ms1)
        ms_90 = self.xr_ar6.sel(ModelScenario=ms1).ModelScenario[(totalneg_i >= totalneg_i.quantile(0.9-0.1)
                                    ).Value & (totalneg_i <= totalneg_i.quantile(0.9+0.1)).Value]
        time_evo_neg_90 = xr_scen2_neg.sel(ModelScenario=ms_90).mean(dim='ModelScenario')
    
        # The 10-percentile
        totalneg_i = totalneg.sel(ModelScenario=ms1)
        ms_10 = self.xr_ar6.sel(ModelScenario=ms1).ModelScenario[(totalneg_i >= totalneg_i.quantile(0.1-0.1)
                                    ).Value & (totalneg_i <= totalneg_i.quantile(0.1+0.1)).Value]
        time_evo_neg_10 = xr_scen2_neg.sel(ModelScenario=ms_10).mean(dim='ModelScenario')

        # Difference and smoothen this
        surplus_factor = emis_all.sel(ModelScenario = np.intersect1d(ms_90, ms1)).mean(dim='ModelScenario').Value - emis_all.sel(ModelScenario = np.intersect1d(ms_10, ms1)).mean(dim='ModelScenario').Value
        wh = np.where(surplus_factor[:30] < 0)[0]
        surplus_factor[wh] = 0
        surplus_factor = savgol_filter(surplus_factor, 40, 3)

        for neg_i, neg in enumerate(self.Neglist):
            xset = emis_all.sel(ModelScenario=ms1)+surplus_factor*(neg-0.5)*2
            factor = (self.xr_co2_budgets.Budget.sel(Temperature=temp, Risk=risk) - xset.sum(dim='Time')) / np.sum(compensation_form)
            all_pathways = (1e3*(xset+factor*xr_comp)).Value/1e3
            if len(all_pathways)>0:
                for unc_i, unc in enumerate(['Earliest', 'Early', 'Medium', 'Late', 'Latest']):
                    ms_path = all_pathways.ModelScenario[np.where(all_pathways.sel(Time=2030) < np.nanpercentile(all_pathways.sel(Time=2030), [10, 25, 50, 75, 90][unc_i], axis=0))[0]]
                    pathway = all_pathways.sel(ModelScenario = ms_path).mean(dim='ModelScenario')#np.array(all_pathways.where(all_pathways.sel(Time=2050) < np.percentile(all_pathways.sel(Time=2050), [10, 25, 50, 75, 90][unc_i], axis=0)).mean(dim='ModelScenario'))#np.percentile(all_pathways, [20, 50, 80][unc_i], axis=0)
                    pathway = savgol_filter(pathway, 20, 3)
                    offset = float(startpoint)/1e3 - pathway[0]
                    pathways_data['CO2_globe'][neg_i, unc_i, temp_i, risk_i, :] = (pathway.T+offset)*1e3
self.xr_traj_co2 = xr_traj_co2.update(pathways_data)

In [ ]:
self.xr_traj_ghg = self.xr_traj_co2+self.xr_traj_nonco2

In [264]:
self.xr_traj_ghg

AttributeError: 'datareading' object has no attribute 'xr_traj_ghg'

In [8]:
import class_allocation

reload(class_allocation)
from class_allocation import allocation

for cty in tqdm(np.array(datareader.xr_total.Region)):
    allocator = allocation(cty)
    allocator.gf()  
    allocator.pc()
    allocator.pcc()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

In [209]:
import class_tempalign
reload(class_tempalign)
from class_tempalign import tempaligning

tempaligner = tempaligning() # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:09<00:00,  1.58s/it]


- Save


In [ ]:
import class_policyscens
reload(class_policyscens)
from class_policyscens import policyscenadding

policyscenner = policyscenadding() # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()